In [7]:
import torch
import torch.nn as nn
import json
import pickle

In [2]:
with open('./MovieReview_SentimentAnalysis-main/top_10000_words.json',encoding='UTF-8') as file:
    data = json.load(file)
    

In [4]:
look_up_table = {word : i for i, word in enumerate(data)}

In [5]:
class BiLSTM_withAT(nn.Module):
    def __init__(self,embeded_size):
        super(BiLSTM_withAT,self).__init__()
        self.embeded_size = embeded_size
        self.embedding_layer = nn.Embedding(self.embeded_size,
                                           embedding_dim = 4)
        self.hidden_size = 64
        self.lstm = nn.LSTM(4,self.hidden_size,bidirectional=True,batch_first=True)
        self.linear = nn.Linear(self.hidden_size*2,1)
        self.sigmoid = nn.Sigmoid()
    def forward(self,x):
        embedding = self.embedding_layer(x)
        lstm_out, (ht,ct) = self.lstm(embedding)
        out = self.linear(lstm_out[:,-1,:])
        return self.sigmoid(out)
        

In [6]:
net = BiLSTM_withAT(len(look_up_table))

In [36]:
with open("./MovieReview_SentimentAnalysis-main/trainDocs.pkl","rb") as file:
    data = pickle.load(file)

In [37]:
data_total = data

In [73]:
train_total = []
for data in data_total:
    item_list = []
    for item in data[0]:
        if item in look_up_table:
            item_list.append(look_up_table[item])
    if(len(item_list)>0):
        train_total.append([item_list,data[1]])
train_x = [x[0] for x in train_total]
train_y = [x[1] for x in train_total]

In [ ]:
epoch = 20
learning_rate = 1e-4
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
lossF = torch.nn.BCELoss()

for i in range(0,epoch):
    for data,label in zip(train_x,train_y):
        data,label = torch.LongTensor([data]),torch.FloatTensor([label])
        y_pred = net(data)
        loss = lossF(y_pred,label)
        optimizer.zero_grad()
        loss.backward()
    print(f'loss : {loss}')